# Grid World迷宫之值循环(Value Iteration)

In [1]:
# 载入相关套件
import numpy as np
from lib.envs.gridworld import GridworldEnv

In [2]:
# 环境
env = GridworldEnv()

In [3]:
# 值循环函数
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    # 计算行动值函数
    def one_step_lookahead(state, V):
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[state][a]:
                A[a] += prob * (reward + discount_factor * V[next_state])
        return A

    # 状态值函数初始化
    V = np.zeros(env.nS)
    while True:
        delta = 0
        # 更新每个状态值的函数
        for s in range(env.nS):
            # 计算下一步的行动值函数
            A = one_step_lookahead(s, V)
            best_action_value = np.max(A)
            # 比较更新前后的差值，取最大值
            delta = max(delta, np.abs(best_action_value - V[s]))
            # 更新状态值函数
            V[s] = best_action_value        
        # 若最大差值 < 门槛值，则停止评估
        if delta < theta:
            break

    # 一开始采随机策略，往上/下/左/右走的机率(π)均等
    policy = np.zeros([env.nS, env.nA])
    for s in range(env.nS):
        # 计算下一步的行动值函数
        A = one_step_lookahead(s, V)
        # 选择最佳行动
        best_action = np.argmax(A)
        # 永远采取最佳行动
        policy[s, best_action] = 1.0
    
    return policy, V            

In [4]:
# 执行值循环
policy, v = value_iteration(env)

In [5]:
# 显示结果
print("策略机率分配:")
print(policy)
print("")

print("4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("状态值函数:")
print(v)
print("")

print("4x4 状态值函数:")
print(v.reshape(env.shape))
print("")

策略机率分配:
[[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]

4x4 策略机率分配 (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

状态值函数:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

4x4 状态值函数:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [6]:
# 验证答案是否正确
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)